In [1]:
from Crypto.Util import number
import time

class RSA:
    def __init__(self) -> None:
        self.e = None
        self.d = None
        self.n = None
        self.p = None
        self.q = None
        self.crt = None
        self.pow=None

    def set_param(self, bit_length):
        self.p = number.getStrongPrime(bit_length)
        self.q = number.getStrongPrime(bit_length)

    def gen_key_pair(self, bit_length):
        bits_pq = int(bit_length / 2)
        self.n = 0
        while self.n.bit_length() != bit_length:
            self.set_param(bits_pq)
            self.n = self.p * self.q
        self.phi_n = (self.p - 1) * (self.q - 1)
        self.e = 0
        while (
            (self.e <= 3)
            | (self.e >= self.phi_n)
            | (number.GCD(self.e, self.phi_n) != 1)
        ):
            self.e = number.getRandomInteger(self.phi_n.bit_length())
        self.d = number.inverse(self.e, self.phi_n)
        self.block_size = len(str(self.n))

    def init_crt(self):
        self.crt = True
        self.dp = self.d % (self.p - 1)
        self.dq = self.d % (self.q - 1)
        self.q_inv = number.inverse(self.q, self.p)

    def sef_pow(self,method):
        self.pow=method
        print(f"将RSA加解密算法的模幂设置为:{self.pow}")

    def get_public_key(self):
        return (self.e, self.n)

    def encrypt(self, m):
        if self.pow=='Montgomery':
            return self.monpow(m, self.e, self.n)
        elif self.pow=='Binary':
            return self.binpow(m, self.e, self.n)
        return pow(m, self.e, self.n)

    def decrypt(self, c):
        if self.pow=='Montgomery':
            return self.monpow(c, self.d, self.n)
        elif self.pow=='Binary':
            return self.binpow(c, self.d, self.n)
        else:
            if self.crt is True:
                m1 = pow(c, self.dp, self.p)
                m2 = pow(c, self.dq, self.q)
                h = (self.q_inv * (m1 - m2)) % self.p
                return m2 + h * self.q
            else:
                return pow(c, self.d, self.n)

    def encrypt_string(self, s):
        ciphertext = ""
        for c in s:
            e_c = str(self.encrypt(ord(c)))
            while len(e_c) < self.block_size:
                e_c = "0" + e_c
            ciphertext += e_c

        return ciphertext

    def decrypt_string(self, s):
        plaintext = ""
        block = ""
        for c in s:
            if len(block) < self.block_size:
                block += c
            else:
                plaintext += chr(self.decrypt(int(block)))
                block = c
        plaintext += chr(self.decrypt(int(block)))
        return plaintext
    
    def binpow(self,a, b, m):
        a = a % m
        res = 1
        while b > 0:
            if (b & 1):
                res = res * a % m
            a = (a * a) % m
            b >>= 1
        return res
    
    def monpow(self,a, b, m):
        c = 1
        a = a % m
        while b > 0:
            if b % 2 == 1:
                c = (c * a) % m
            b = b >> 1
            a = (a * a) % m
        return c


In [2]:
m = "2024 Cryptographic Engineering Principle and Practice"
print(f"加密前字符串：{m}")
rsa = RSA()
rsa.gen_key_pair(1024)
c = rsa.encrypt_string(m)
print(f"加密后字符串：{c}")

加密前字符串：2024 Cryptographic Engineering Principle and Practice
加密后字符串：07980417877426346401260336698221062309216227134191411595876394530226792604316566775431593234381255706256607048556976109255528042705194448274880858734559670918228687851441140384681411761149580812660517341029486138166904118266808972263847043784249720592671951510000349361539797245000252690557513362155124457624709863128421394195252162434405229369324518939757557814881653031399793680401482343409734215300569491751909324707744606485797494406456215818213037294980067081580728797581633822247705960707643964001237379532202454555340506844820860463396218322034298013653926816374644614649031456360069083217614407895404829096006907980417877426346401260336698221062309216227134191411595876394530226792604316566775431593234381255706256607048556976109255528042705194448274880858734559670918228687851441140384681411761149580812660517341029486138166904118266808972263847043784249720592671951510000349361539797245000252690557513362155124457624713960

In [3]:
'''标准RSA算法'''
start_time = time.perf_counter()
ec = rsa.decrypt_string(c)
end_time = time.perf_counter()
print(f"解密后字符串：{ec}")
print("Execution time: {:.6f} milliseconds".format((end_time - start_time)*1000))

解密后字符串：2024 Cryptographic Engineering Principle and Practice
Execution time: 149.819593 milliseconds


In [4]:
'''使用CRT进行解密'''
rsa.init_crt()
start_time = time.perf_counter()
ec=rsa.decrypt_string(c)
end_time = time.perf_counter()
print(f"解密后字符串：{ec}")
print("Execution time: {:.6f} milliseconds".format((end_time - start_time)*1000))

解密后字符串：2024 Cryptographic Engineering Principle and Practice
Execution time: 49.269965 milliseconds


In [5]:
'''使用python中的pow模幂'''
rsa.sef_pow('pow')
start_time = time.perf_counter()
c = rsa.encrypt_string(m)
ec = rsa.decrypt_string(c)
end_time = time.perf_counter()
print(f"解密后字符串：{ec}")
print("Execution time: {:.6f} milliseconds".format((end_time - start_time)*1000))

将RSA加解密算法的模幂设置为:pow
解密后字符串：2024 Cryptographic Engineering Principle and Practice
Execution time: 173.472628 milliseconds


In [6]:
'''使用Montgomery模幂'''
rsa.sef_pow('Montgomery')
start_time = time.perf_counter()
c = rsa.encrypt_string(m)
ec = rsa.decrypt_string(c)
end_time = time.perf_counter()
print(f"解密后字符串：{ec}")
print("Execution time: {:.6f} milliseconds".format((end_time - start_time)*1000))

将RSA加解密算法的模幂设置为:Montgomery
解密后字符串：2024 Cryptographic Engineering Principle and Practice
Execution time: 396.983201 milliseconds


In [7]:
'''使用Binary模幂'''
rsa.sef_pow('Binary')
start_time = time.perf_counter()
c = rsa.encrypt_string(m)
ec = rsa.decrypt_string(c)
end_time = time.perf_counter()
print(f"解密后字符串：{ec}")
print("Execution time: {:.6f} milliseconds".format((end_time - start_time)*1000))

将RSA加解密算法的模幂设置为:Binary
解密后字符串：2024 Cryptographic Engineering Principle and Practice
Execution time: 391.611945 milliseconds
